In [1]:
cd home/my-project

bash: cd: home/my-project: No such file or directory


: 1

# 1.4 Merging Branches

## In practice:
In this section, we'll integrate work from `new-branch` back into `main`, exploring different merge strategies.

## Types of Merges
Understanding the different types of merges in Git is crucial for effective branch management.

### Fast-forward merge
This merge strategy is used when the target branch can be updated to point to the commit of the source branch without needing to create a new commit.

### Three-way merge
A three-way merge is used when the two branches have diverged and a new commit is required to reconcile the differences.

## Discussion: How do different merges work?

### Fast-forward merge
To merge two branches, we use `git merge <branch-name>` while in the branch we want to merge into. For a fast-forward merge:

In [2]:
git switch main
git merge new-branch
git status
git log

D	_build/.doctrees/environment.pickle
M	_toc.yml
D	ch1/file1.txt
D	ch11.md
D	ch12.md
D	ch13.md
D	ch14.md
D	ch15.md
D	ch1_introduction_to_version_control.md
M	requirements.txt
Switched to branch 'main'
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)
Updating 089f7ed..b463777
error: The following untracked working tree files would be overwritten by merge:
	ch1/ch11.md
	ch1/ch12.md
	ch1/ch13.md
	ch1/ch14.md
	ch1/ch15.md
	ch1/ch1_introduction_to_version_control.md
Please move or remove them before you merge.
Aborting
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    ../_build/.doctrees/environment.pickle
	modified:   ../_toc.yml
	deleted:    file1.txt
	deleted:    ../ch11.md
	deleted:    ../ch12.m

M	_toc.yml


D	ch1/file1.txt


D	ch11.md


D	ch12.md


...

    Add third line to file1.txt


commit 188ad45f8500fad34cea2ec76a65811217ea23f0 (origin/main)


Author: mcallara <mcallara@gmail.com>


Date:   Wed Feb 7 15:48:53 2024 +0100


    initial version


```{admonition} What to notice
:class: hint
The `HEAD` of `main` is now pointing to the same commit as `new-branch`.
```

Switching branches may change the working directory, especially if they point to different commits.

## Exercise: Switching and Committing

In [3]:
git switch new-branch
touch file2.txt
echo -n "This is file 2." >> file2.txt
echo -en "\nThis is the fourth line." >> file1.txt
git add .
git commit -m "Add file2 and modify file1"
git switch main

error: The following untracked working tree files would be overwritten by checkout:
	ch1/ch11.md
	ch1/ch12.md
	ch1/ch13.md
	ch1/ch14.md
	ch1/ch15.md
	ch1/ch1_introduction_to_version_control.md
Please move or remove them before you switch branches.
Aborting
[main e91a414] Add file2 and modify file1
 8 files changed, 660 insertions(+), 2 deletions(-)
 create mode 100644 ch1/ch11.md
 create mode 100644 ch1/ch12.md
 create mode 100644 ch1/ch13.md
 create mode 100644 ch1/ch14.md
 create mode 100644 ch1/ch15.md
 create mode 100644 ch1/ch1_introduction_to_version_control.md
 create mode 100644 ch1/file2.txt
D	_build/.doctrees/environment.pickle
M	_toc.yml
D	ch11.md
D	ch12.md
D	ch13.md
D	ch14.md
D	ch15.md
D	ch1_introduction_to_version_control.md
M	requirements.txt
Already on 'main'
Your branch is ahead of 'origin/main' by 2 commits.
  (use "git push" to publish your local commits)


	ch1/ch11.md


	ch1/ch12.md


	ch1/ch13.md


	ch1/ch14.md


...

D	ch1_introduction_to_version_control.md


M	requirements.txt


Already on 'main'


Your branch is ahead of 'origin/main' by 2 commits.


  (use "git push" to publish your local commits)


```{admonition} What to notice
:class: hint
The working directory content changes when switching branches.
```

Git prevents loss of work by disallowing branch switching with uncommitted changes.

## Exercise: Preventing Data Loss

In [4]:
git switch new-branch
echo -en "\nThis is the fifth line." >> file1.txt
git switch main

D	_build/.doctrees/environment.pickle
M	_toc.yml
D	ch11.md
D	ch12.md
D	ch13.md
D	ch14.md
D	ch15.md
D	ch1_introduction_to_version_control.md
M	requirements.txt
Switched to branch 'new-branch'
error: Your local changes to the following files would be overwritten by checkout:
	ch1/file1.txt
Please commit your changes or stash them before you switch branches.
Aborting


M	_toc.yml


D	ch11.md


D	ch12.md


D	ch13.md


...

error: Your local changes to the following files would be overwritten by checkout:


	ch1/file1.txt


Please commit your changes or stash them before you switch branches.


Aborting


: 1

```{admonition} What to notice
:class: hint
Git will not allow branch switching with uncommitted changes to protect against data loss.
```

## Note on Staging
Changes made after staging are not automatically included in the staged snapshot.

## Exercise: Staging and Modifying

In [5]:
git add file1.txt
echo -en "\nThis is the sixth line." >> file1.txt
git add file1.txt
git commit -m "Add fifth and sixth lines to file1.txt"

[new-branch 6fe4564] Add fifth and sixth lines to file1.txt
 1 file changed, 2 insertions(+)


 1 file changed, 2 insertions(+)


## Viewing All Commits

In [6]:
git switch main
git log --all

D	_build/.doctrees/environment.pickle
M	_toc.yml
D	ch11.md
D	ch12.md
D	ch13.md
D	ch14.md
D	ch15.md
D	ch1_introduction_to_version_control.md
M	requirements.txt
Switched to branch 'main'
Your branch is ahead of 'origin/main' by 2 commits.
  (use "git push" to publish your local commits)
commit e91a414cd6a277c2b1f1d62864f7c6cdf8ad2cad (HEAD -> main)
Author: mcallara <mcallara@gmail.com>
Date:   Thu Feb 8 09:07:15 2024 +0100

    Add file2 and modify file1

commit 6fe4564de0879304221f9138421519756fbaa218 (new-branch)
Author: mcallara <mcallara@gmail.com>
Date:   Thu Feb 8 09:07:15 2024 +0100

    Add fifth and sixth lines to file1.txt

commit b463777031e9afefdd768b100da12deca5dbdcdd
Author: mcallara <mcallara@gmail.com>
Date:   Thu Feb 8 09:02:05 2024 +0100

    Add fifth and sixth lines to file1.txt

commit 05afa652d76eefa2983d990e713636268cff76ac
Author: mcallara <mcallara@gmail.com>
Date:   Thu Feb 8 09:02:05 2024 +0100

    Add file2 and modify file1

commit 089f7ed655b98fd45b9ea9d2704

M	_toc.yml


D	ch11.md


D	ch12.md


D	ch13.md


...

    Update documentation


commit 188ad45f8500fad34cea2ec76a65811217ea23f0 (origin/main)


Author: mcallara <mcallara@gmail.com>


Date:   Wed Feb 7 15:48:53 2024 +0100


    initial version


```{admonition} What to notice
:class: hint
Commits from `new-branch` are visible with `git log --all`, showing the comprehensive history across branches.
```

## Detached HEAD State
Exploring commits directly can lead to a detached HEAD state, which means you're no longer working on the tip of a branch.

## Creating and Switching Branches
Git offers shortcuts to create a new branch and switch to it in one operation, enhancing workflow efficiency.

In [7]:
git switch -c new-feature-branch

fatal: A branch named 'new-feature-branch' already exists.


: 128

This chapter provides practical exercises to understand merging strategies, branch management, and Git's protections against data loss.